In [49]:
#!ls -lah /cellar/users/btsui/all_seq_snp/Homo_sapiens_all_merged_snp.pickle.gz

In [50]:
import pandas as pd
import numpy as np
import re
import os
import math
from multiprocessing import Pool
from tqdm import tqdm
## init
#/nrnb/users/btsui/Data/tcga_extracted_lgg_snp/
mySpecie='Homo_sapiens'
outMergedDir='/cellar/users/btsui/all_seq_snp/'+mySpecie+'_all_merged_snp.TCGA.with_pcr_rm.pickle'

##change this dir to point to the updated csv
#full_meta_dir="/cellar/users/btsui/Project/METAMAP/notebook/Parsing/sra_dump.csv"
inSrrDir='/nrnb/users/btsui/Data/tcga_extracted_lgg_snp/'
tmp_dir='/nrnb/users/btsui/Data/all_seq/tmp_tcga_block/'

In [51]:
#!ls /nrnb/users/btsui/Data/all_seq/tmp_tcga_block/

In [52]:
tmpBedDf=pd.read_csv('/data/cellardata/users/btsui/dbsnp/snp_beds/'+mySpecie+'.bed',header=None,sep='\t')
unique_chroms=tmpBedDf[0].astype(np.str).unique()

### start merging one by one 
if os.path.exists(tmp_dir):
    os.system('rm -r '+tmp_dir)
os.system('mkdir -p '+tmp_dir)



/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0

In [53]:
snp_fname_postfix='.snp.txt.gz'

In [54]:
os.chdir(tmp_dir)
#identify non empty files
os.system('ls -la '+inSrrDir+' > ls_out.txt ')
ls_df=pd.read_csv('ls_out.txt',sep='\s+',header=None,names=np.arange(9)).iloc[1:]
#ls_df=
size_S=ls_df[4]
m4=size_S.astype(np.int)>1000
m5=ls_df[8].str.contains(snp_fname_postfix+'$')
non_empty_files=ls_df[m4&m5][8].str.split('/').str[-1].str.split('.').str[0].values



In [55]:
def parseSrr(inSrr):
    #print (inSrr)
    fname=inSrrDir+inSrr+snp_fname_postfix
    #print(fname)
    tmpDf_all=pd.read_csv(fname,sep='\s+',header=None,names=np.arange(50),index_col=None,error_bad_lines=False)
    #print (tmpDf_all.head())
    myCols=['Chr','Pos','Ref','rd_all','','A','C','G','T','N']
    tmpDf=tmpDf_all.iloc[:,:len(myCols)]
    tmpDf.columns=myCols
    tmpDf2=tmpDf.set_index(['Chr','Pos'])
    myBases=['A','C','G','T']
    myL=[]
    for base in myBases:
            splitL=tmpDf2[base].str.split(':',expand=True)
            ### extract  the read count and base quality
            tmpDf5=splitL[[1,3]].astype(np.float)
            tmpDf5.columns=['ReadDepth','AverageBaseQuality']
            myL.append(tmpDf5)
    tmpDf6=pd.concat(myL,keys=myBases,axis=0,names=['base'])
    tmpDf6.columns.name='features'
    mergedDf=tmpDf6.astype(np.uint16)
    non_zero_df=mergedDf[mergedDf['ReadDepth']>0]
    tmpDf7=non_zero_df.reset_index()
    #Run_digits=re.search('[DES]RR(\d+)', inSrr)
    #Run_Db=re.search('([DES]RR)\d+', inSrr)
    tmpDf7['Run_digits']=inSrr
    tmpDf7['Run_db']='TCGA'
    ###convert the datatypes
    tmpDf7['Pos']=tmpDf7['Pos'].astype(np.uint32)    
    #tmpDf7['Run_digits']=tmpDf7['Run_digits'].astype(np.uint64)
    tmpDf7['Chr']=tmpDf7['Chr'].astype(np.str).astype('category',
                                                      categories=unique_chroms,ordered=True)
    
    #tmpDf7['Run_db']=tmpDf7['Run_db'].astype(np.str).astype('category',
    #                                                        categories=['DRR','ERR','SRR'],ordered=True)
    tmpDf7['base']=tmpDf7['base'].astype('category',
                                         categories=myBases,ordered=True)
    groupings_L=['Run_db','Run_digits',u'Chr', u'Pos',u'base']
    tmpDf7=tmpDf7.drop_duplicates(groupings_L)
    srr_pickle_df=tmpDf7.set_index(groupings_L).sort_index()
    return srr_pickle_df


In [56]:
"""
given: srr id 
return: the merged file
"""

### identify files to be merged
fnames=pd.Series(os.listdir(inSrrDir))
snpFnames=fnames[fnames.str.contains(snp_fname_postfix+'$')]
srrsWithData=snpFnames.str.split('.').str[0]

### configure for loading data

In [57]:
m3=srrsWithData.isin(non_empty_files)
toMergeSrrs=srrsWithData[m3].values
print ('n files to merge: ',len(toMergeSrrs) )
TEST=False
if TEST:
    toRunSrrs=toMergeSrrs[:10]
    chunkSize=5
    nThread=1
else:
    toRunSrrs=toMergeSrrs
    chunkSize=10
    nThread=16

n files to merge:  452


### merge each chunkSize amount of VCFs  among into different pickle chunks

In [58]:
#optional: free up the memory
#if not TEST:
#    del mySpecieDf, full_meta_df

def mergeSrrsL(i):
    tmpL=[]
    failedSrrsL=[]
    for srr in toRunSrrs[i:(i+chunkSize)]:
        ##try:
        tmpL.append(parseSrr(srr))
        #except :
        #print ('failed: '+srr)
        failedSrrsL.append(srr)
    tmpMergedDf=pd.concat(tmpL)
    #tmpMergedDf=pd.concat([parseSrr(srr) for srr in toRunSrrs[i:(i+chunkSize)]])
    reorderedDf=tmpMergedDf.sort_index()
    reorderedDf.to_pickle(tmp_dir+str(i)+'.pickle.gz',compression='gzip')
    return failedSrrsL

Chunks=np.arange(0, len(toRunSrrs),chunkSize)
if TEST:
    failed_srr_l=list(map(mergeSrrsL,Chunks.tolist()))
else:
    from multiprocessing import Pool
    p=Pool(nThread)
    ### sweep for uncompleted chunks
    failed_srr_l=tqdm(list(p.imap(mergeSrrsL,Chunks.tolist())),total=len(toRunSrrs))
    p.close()

/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns 

/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:37: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/cellar/users/btsui/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:32: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
/cellar/users/btsui/

/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,12) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,11) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  self._target(*self._args, **self._kwargs)
/cellar/users/btsui/anaconda3/lib/python3.6/multiprocessing/process.py:93: DtypeWarning: Columns (0,

In [59]:
myL=[]
for fname in tqdm(os.listdir(tmp_dir)):
    if '.pickle.gz' in fname:
        myL.append(pd.read_pickle(tmp_dir+fname))





  0%|          | 0/47 [00:00<?, ?it/s]



  2%|▏         | 1/47 [00:00<00:12,  3.81it/s]



  4%|▍         | 2/47 [00:00<00:11,  3.99it/s]



  6%|▋         | 3/47 [00:05<01:15,  1.71s/it]



  9%|▊         | 4/47 [00:05<00:58,  1.36s/it]



 11%|█         | 5/47 [00:05<00:47,  1.14s/it]



 13%|█▎        | 6/47 [00:05<00:40,  1.01it/s]



 15%|█▍        | 7/47 [00:06<00:35,  1.14it/s]



 17%|█▋        | 8/47 [00:06<00:34,  1.14it/s]



 21%|██▏       | 10/47 [00:07<00:26,  1.38it/s]



 23%|██▎       | 11/47 [00:07<00:25,  1.42it/s]



 26%|██▌       | 12/47 [00:08<00:23,  1.49it/s]



 28%|██▊       | 13/47 [00:08<00:21,  1.55it/s]



 30%|██▉       | 14/47 [00:09<00:21,  1.53it/s]



 32%|███▏      | 15/47 [00:09<00:19,  1.60it/s]



 34%|███▍      | 16/47 [00:09<00:18,  1.67it/s]



 36%|███▌      | 17/47 [00:10<00:17,  1.70it/s]



 38%|███▊      | 18/47 [00:10<00:16,  1.76it/s]



 40%|████      | 19/47 [00:10<00:15,  1.81it/s]



 43%|████▎     | 20/47 [00:10<00:14,  1.85it

In [60]:
mergedDf=pd.concat(myL,axis=0)

In [61]:
#mergedDf

In [62]:
mergedDf.to_pickle(outMergedDir)

In [63]:
#!ls -lah /cellar/users/btsui/all_seq_snp/Homo_sapiens_all_merged_snp.TCGA.pickle

In [64]:
outMergedDir

'/cellar/users/btsui/all_seq_snp/Homo_sapiens_all_merged_snp.TCGA.with_pcr_rm.pickle'

In [65]:
outMergedDir

'/cellar/users/btsui/all_seq_snp/Homo_sapiens_all_merged_snp.TCGA.with_pcr_rm.pickle'